In [16]:
# I know ... this is a complete mess. I'm deeply sorry. I'll not clean it up later.

In [1]:
import math
import numpy as np
from numpy.typing import NDArray
import pandas as pd
import swifter
import pickle
from typing import Union, List, Any, Tuple
from qaoa_and_tdvp import QAOA, tdvp_optimize_qaoa, scipy_optimize, QAOAResult, Qobj
from MaxCut import MaxCut
from qutip import tensor, basis, Qobj, expect, variance

from benchmark import (
    get_all_connected,
    get_connected_rn_graph,
    get_rn_qubo,
    select_if_connected,
    Benchmark,
load_instances,
)
swifter;MaxCut;get_all_connected; get_connected_rn_graph; get_rn_qubo; select_if_connected; Benchmark; # noinspection PyStatementEffect

In [4]:
with open('results/data.p','rb') as f:
    data = pickle.load(f)

In [5]:
with open("results/tdvp_gd.p",'rb') as file:
    tdvp_gd= pickle.load(file)

In [55]:
tdvp_gd.loc[4,3,4]['gradient_descent']

algorithm                                                           gradient_descent
delta                              (5.576299447028864, 15.71813582815118, 7.50804...
delta_0                                                           (1, 1, 1, 1, 1, 1)
instance                                    <MaxCut.MaxCut object at 0x7f592ac867d0>
message                            Gradient descent solver terminated with       ...
n                                                                                  4
num_fun_calls                                                                   None
p                                                                                  3
path                               [(2.1921361675604203, 1.4088018812353766, 0.56...
real duration                                                             121.987031
res                                \n        gradient_descent with general gradie...
steps                                                            

In [53]:
data['tdvp'].loc[4,3,4]

instance                                    <MaxCut.MaxCut object at 0x7f592ac20280>
p                                                                                  3
n                                                                                  4
delta_0                                                           (1, 1, 1, 1, 1, 1)
tollarance                                                                      0.01
algorithm                                                                       tdvp
res                                \n        tdvp_optimizer with circuit gradient...
delta                              (0.9105912868139331, 0.7990204517949632, 0.332...
value                                                                      -8.153389
path                               [(1.0, 1.0, 1.0, 1.0, 1.0, 1.0), (1.1564780346...
steps                                                                             29
num_fun_calls                                                    

In [54]:
for idx in tdvp_gd.index:
    datapoint = data['tdvp'].loc[idx]
    gd_steps = tdvp_gd['gradient_descent'].loc[idx].steps
    qaoa = QAOA(datapoint.instance.qubo, p=datapoint.p)
    try:
        datapoint.path = datapoint.path[:gd_steps]
        print(f"pruned {datapoint.steps - gd_steps} steps in {idx}")
        datapoint.steps = len(datapoint.path)
        datapoint.value = qaoa.expectation(datapoint.path[-1])
    except TypeError:
        print(f'skipping {idx}. tdvp.sucess = {datapoint.success}')
        continue
    tdvp_gd.loc[idx]['tdvp'] = pd.Series(data=compute_measures(datapoint, qaoa), index=datapoint.index)

pruned -51 steps in (3, 1, 0)
pruned -50 steps in (3, 1, 1)
pruned -83 steps in (3, 2, 0)
pruned -78 steps in (3, 2, 1)
pruned -48 steps in (4, 1, 0)
pruned -47 steps in (4, 1, 1)
pruned -46 steps in (4, 1, 2)
pruned -46 steps in (4, 1, 3)
pruned -46 steps in (4, 1, 4)
pruned -49 steps in (4, 1, 5)
pruned -178 steps in (4, 2, 0)
pruned -182 steps in (4, 2, 1)
pruned -171 steps in (4, 2, 2)
pruned -185 steps in (4, 2, 3)
pruned -172 steps in (4, 2, 4)
pruned -107 steps in (4, 2, 5)
pruned -538 steps in (4, 3, 0)
pruned -136 steps in (4, 3, 1)
pruned -523 steps in (4, 3, 2)
pruned -504 steps in (4, 3, 3)
pruned -519 steps in (4, 3, 4)
skipping (4, 3, 5). tdvp.sucess = False
pruned -67 steps in (4, 4, 0)
pruned -67 steps in (4, 4, 1)
pruned -56 steps in (4, 4, 2)
skipping (4, 4, 3). tdvp.sucess = False
pruned -67 steps in (4, 4, 4)
skipping (4, 4, 5). tdvp.sucess = False
skipping (4, 5, 0). tdvp.sucess = False
pruned -194 steps in (4, 5, 1)
pruned -349 steps in (4, 5, 2)
skipping (4, 5, 3

In [2]:
with open('results/success_count.p','rb') as f:
    success_count = pickle.load(f)

In [3]:
success_count

,1,2,3,4,5
tdvp,"{'count': 18, 'total': 18, 'ratio': 1.0}","{'count': 17, 'total': 18, 'ratio': 0.94444444...","{'count': 12, 'total': 16, 'ratio': 0.75}","{'count': 10, 'total': 16, 'ratio': 0.625}","{'count': 3, 'total': 6, 'ratio': 0.5}"
scipy,"{'count': 18, 'total': 18, 'ratio': 1.0}","{'count': 18, 'total': 18, 'ratio': 1.0}","{'count': 16, 'total': 16, 'ratio': 1.0}","{'count': 16, 'total': 16, 'ratio': 1.0}","{'count': 6, 'total': 6, 'ratio': 1.0}"
gradient_descent,"{'count': 16, 'total': 18, 'ratio': 0.88888888...","{'count': 8, 'total': 18, 'ratio': 0.444444444...","{'count': 0, 'total': 16, 'ratio': 0.0}","{'count': 0, 'total': 16, 'ratio': 0.0}","{'count': 0, 'total': 6, 'ratio': 0.0}"


In [225]:
success_count = {
    p: {
        "tdvp": {
            "count": 0,
            "total": 0,
            'ratio': 0,
        },
        "scipy": {
            "count": 0,
            "total": 0,
            'ratio': 0,
        },
        "gradient_descent": {
            "count": 0,
            "total": 0,
            'ratio': 0,
        },
    }
    for p in range(1, 6)
}

for idx in data.index:
    p = idx[1]
    for alg in ["tdvp", "scipy", "gradient_descent"]:
        entry = success_count[p][alg]
        entry["total"] += 1
        if data.loc[idx][alg]["success"]:
            entry["count"] += 1
        entry["ratio"] = entry["count"] / entry["total"]


In [227]:
# with open('results/success_count.p','wb') as f:
#     pickle.dump(pd.DataFrame(success_count), f)


# Combina data in one frame

In [2]:
with open('./results/tdvp_vs_gd_n3_new.p','rb') as f:
    data = pickle.load(f)

In [16]:
with open("./results/stats.p",'rb') as file:
    stats = pickle.load(file)

In [3]:
def f(x):
    x['num_fun_calls'] =4*x['p']*x['steps']
    return x

In [3]:
data

instance  \
p i                                             
1 0  <MaxCut.MaxCut object at 0x7fbb4ee01060>   
  1  <MaxCut.MaxCut object at 0x7fbb4ee02770>   
2 0  <MaxCut.MaxCut object at 0x7fbb4ee01060>   
  1  <MaxCut.MaxCut object at 0x7fbb4ee02770>   

                                                  tdvp  \
p i                                                      
1 0  algorithm        NaN
delta            NaN
delt...   
  1  algorithm        NaN
delta            NaN
delt...   
2 0  algorithm        NaN
delta            NaN
delt...   
  1  algorithm        NaN
delta            NaN
delt...   

                                                 scipy  \
p i                                                      
1 0  algorithm        NaN
delta            NaN
delt...   
  1  algorithm        NaN
delta            NaN
delt...   
2 0  algorithm        NaN
delta            NaN
delt...   
  1  algorithm        NaN
delta            NaN
delt...   

                                      gradient_descent  
p i                                                     
1 0  algorithm                                     ...  
  1  algorithm                                     ...  
2 0  algorithm                                     ...  
  1  algorithm                                     ...

In [12]:
for idx in data.index:
    data.loc[idx]['gradient_descent']['num_fun_calls'] = 4*data.loc[idx]['gradient_descent']['p']*data.loc[idx]['gradient_descent']['steps']

In [16]:
data.loc[3,1,0]['gradient_descent']

instance                                    <MaxCut.MaxCut object at 0x7f069a2e0df0>
p                                                                                  1
n                                                                                  3
delta_0                                                                       (1, 1)
tollarance                                                                      0.01
algorithm                                                           gradient_descent
res                                \n        gradient_descent with general gradie...
delta                                       (0.7774068338785014, 0.4461099550591143)
value                                                                      -3.006697
path                               [(0.9949501838791832, 0.9575015285913082), (0....
steps                                                                             56
num_fun_calls                                                    

In [8]:
stats

expectation approximation ratio                      \
                                                mean       std       min   
n Algorithm        p                                                       
3 tdvp             1                        0.911088  0.073714  0.858965   
                   2                         0.93039  0.063845  0.885245   
  scipy            1                        0.778976  0.112656  0.699316   
                   2                        0.930466  0.058679  0.888974   
  gradient_descent 1                        0.911161  0.073687  0.859056   
                   2                        0.942691   0.04645  0.909846   
4 tdvp             1                        0.555697  0.320149  0.133846   
                   2                        0.914538  0.023949  0.873142   
                   3                        0.926812  0.027653  0.895857   
                   4                        0.943432  0.049538  0.869999   
                   5                        0.969683  0.032257  0.935768   
  scipy            1                        0.578786  0.215603   0.32276   
                   2                        0.850797  0.141596   0.56292   
                   3                        0.880351   0.07396  0.760516   
                   4                        0.832498  0.142717   0.62381   
                   5                        0.941948  0.074239  0.821507   
  gradient_descent 1                        0.515058  0.413939  0.067843   
                   2                         0.91065  0.025242  0.874111   
                   3                             NaN       NaN       NaN   
                   4                             NaN       NaN       NaN   
                   5                             NaN       NaN       NaN   
5 tdvp             1                        0.368847  0.280092 -0.001885   
                   2                         0.84504  0.163872  0.513454   
                   3                        0.879854  0.119161  0.667211   
                   4                         0.78599  0.095736  0.678714   
  scipy            1                        0.502557  0.245925  0.096577   
                   2                        0.606018  0.267695  0.200558   
                   3                        0.730393  0.186023  0.493616   
                   4                        0.787536   0.13729  0.588533   
  gradient_descent 1                        0.254755  0.301218  -0.00173   
                   2                        0.701179  0.265325  0.513566   
                   3                             NaN       NaN       NaN   
                   4                             NaN       NaN       NaN   

                               groundspace overlap                      \
                           max                mean       std       min   
n Algorithm        p                                                     
3 tdvp             1  0.963212            0.869012  0.145009  0.766475   
                   2  0.975536            0.931829  0.071453  0.881304   
  scipy            1  0.858635            0.766332  0.006235  0.761923   
                   2  0.971958             0.89338  0.117401  0.810365   
  gradient_descent 1  0.963266            0.867664   0.14638  0.764157   
                   2  0.975536            0.912558  0.098755  0.842728   
4 tdvp             1  0.963225            0.464344   0.37313  0.019871   
                   2  0.947408            0.895209  0.049664  0.831052   
                   3  0.966206            0.866436   0.06825  0.795922   
                   4  0.974706            0.930693  0.062414  0.843536   
                   5  0.999978            0.960623  0.048394  0.906591   
  scipy            1   0.79544            0.385312  0.334051   0.01563   
                   2   0.91854            0.786808  0.231791  0.335994   
                   3  0.973016            0.816939  0.087629  0.752143   
                   4  0.98

In [12]:
tdvp_stats = stats.xs('tdvp',level='Algorithm')
tdvp_stats['num_fun_calls']

mean          std   min     max
n p                                       
3 1        59.0     4.242641  56.0    62.0
  2        95.0    29.698485  74.0   116.0
4 1        55.0     7.014271  44.0    62.0
  2       194.0   203.399115  56.0   602.0
  3  548.666667  1003.912081   0.0  2588.0
  4   83.333333    71.079298   0.0   182.0
  5       370.0   621.564478   0.0  1574.0
5 1        55.4     22.52998  20.0    92.0
  2       184.2   152.561682   0.0   542.0
  3       110.0    88.528715   0.0   224.0
  4       314.0   650.809411   0.0  2134.0

In [56]:
# with open('./results/n4_data.p', 'rb') as f:
#     n4 = pickle.load(f)

In [57]:
# n4['gradient_descent'] = n4_grad['gradient_descent']
# df = n4
# df

instance  \
p i                                             
1 0  <MaxCut.MaxCut object at 0x7f8c49901480>   
  1  <MaxCut.MaxCut object at 0x7f8c48e55600>   
  2  <MaxCut.MaxCut object at 0x7f8c42e7a230>   
  3  <MaxCut.MaxCut object at 0x7f8c42e780a0>   
  4  <MaxCut.MaxCut object at 0x7f8c42e7a800>   
  5  <MaxCut.MaxCut object at 0x7f8c42e7a6e0>   
2 0  <MaxCut.MaxCut object at 0x7f8c49901480>   
  1  <MaxCut.MaxCut object at 0x7f8c48e55600>   
  2  <MaxCut.MaxCut object at 0x7f8c42e7a230>   
  3  <MaxCut.MaxCut object at 0x7f8c42e780a0>   
  4  <MaxCut.MaxCut object at 0x7f8c42e7a800>   
  5  <MaxCut.MaxCut object at 0x7f8c42e7a6e0>   
3 0  <MaxCut.MaxCut object at 0x7f8c49901480>   
  1  <MaxCut.MaxCut object at 0x7f8c48e55600>   
  2  <MaxCut.MaxCut object at 0x7f8c42e7a230>   
  3  <MaxCut.MaxCut object at 0x7f8c42e780a0>   
  4  <MaxCut.MaxCut object at 0x7f8c42e7a800>   
  5  <MaxCut.MaxCut object at 0x7f8c42e7a6e0>   
4 0  <MaxCut.MaxCut object at 0x7f8c49901480>   
  1  <MaxCut.MaxCut object at 0x7f8c48e55600>   
  2  <MaxCut.MaxCut object at 0x7f8c42e7a230>   
  3  <MaxCut.MaxCut object at 0x7f8c42e780a0>   
  4  <MaxCut.MaxCut object at 0x7f8c42e7a800>   
  5  <MaxCut.MaxCut object at 0x7f8c42e7a6e0>   
5 0  <MaxCut.MaxCut object at 0x7f8c49901480>   
  1  <MaxCut.MaxCut object at 0x7f8c48e55600>   
  2  <MaxCut.MaxCut object at 0x7f8c42e7a230>   
  3  <MaxCut.MaxCut object at 0x7f8c42e780a0>   
  4  <MaxCut.MaxCut object at 0x7f8c42e7a800>   
  5  <MaxCut.MaxCut object at 0x7f8c42e7a6e0>   

                                                  tdvp  \
p i                                                      
1 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   
2 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   
3 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   
4 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   
5 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   

                                                 scipy  \
p i                                                      
1 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   
2 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                  

# convert data into DataFrame

In [25]:
with open('./results/n5_data.p', 'rb') as file:
    df = pickle.load(file)

df.head(10)

instance  \
p i                                             
1 0  <MaxCut.MaxCut object at 0x7f8c49820970>   
  1  <MaxCut.MaxCut object at 0x7f8c498207c0>   
  2  <MaxCut.MaxCut object at 0x7f8c498233a0>   
  3  <MaxCut.MaxCut object at 0x7f8c49822530>   
  4  <MaxCut.MaxCut object at 0x7f8c49823610>   
  5  <MaxCut.MaxCut object at 0x7f8c498237f0>   
  6  <MaxCut.MaxCut object at 0x7f8c498239a0>   
  7  <MaxCut.MaxCut object at 0x7f8c49823b80>   
  8  <MaxCut.MaxCut object at 0x7f8c49823d60>   
  9  <MaxCut.MaxCut object at 0x7f8c49821450>   

                                                  tdvp  \
p i                                                      
1 0  instance                  <MaxCut.MaxCut objec...   
  1  instance                  <MaxCut.MaxCut objec...   
  2  instance                  <MaxCut.MaxCut objec...   
  3  instance                  <MaxCut.MaxCut objec...   
  4  instance                  <MaxCut.MaxCut objec...   
  5  instance                  <MaxCut.MaxCut objec...   
  6  instance                  <MaxCut.MaxCut objec...   
  7  instance                  <MaxCut.MaxCut objec...   
  8  instance                  <MaxCut.MaxCut objec...   
  9  instance                  <MaxCut.MaxCut objec...   

                                                 scipy  \
p i                                                      
1 0  instance                  <MaxCut.MaxCut objec...   
  1  instance                  <MaxCut.MaxCut objec...   
  2  instance                  <MaxCut.MaxCut objec...   
  3  instance                  <MaxCut.MaxCut objec...   
  4  instance                  <MaxCut.MaxCut objec...   
  5  instance                  <MaxCut.MaxCut objec...   
  6  instance                  <MaxCut.MaxCut objec...   
  7  instance                  <MaxCut.MaxCut objec...   
  8  instance                  <MaxCut.MaxCut objec...   
  9  instance                  <MaxCut.MaxCut objec...   

                                      gradient_descent  
p i                                                     
1 0  algorithm                                     ...  
  1  algorithm                                     ...  
  2  algorithm                                     ...  
  3  algorithm                                     ...  
  4  algorithm                                     ...  
  5  algorithm                                     ...  
  6  algorithm                                     ...  
  7  algorithm                                     ...  
  8  algorithm                                     ...  
  9  algorithm                                     ...

In [58]:
# def empty_result(res:float|pd.Series)->pd.Series:
#     if isinstance(res, pd.Series):
#         return res
#     else:
#         ser = pd.Series({
#             "algorithm": 'gradient_descent',
#             "instance": None,
#             # "qaoa": qaoa,   # produces unnecessary large files
#             "p": np.NaN,
#             "n": np.NaN,
#             "delta_0": np.NaN,
#             "tollarance": np.NaN,
#             "res": np.NaN,
#             "delta": np.NaN,
#             "value": np.NaN,
#             "path": np.NaN,
#             "steps": np.NaN,
#             "num_fun_calls": np.NaN,
#             "real duration": np.NaN,
#             "message": '',
#         })
#         for meas in measures:
#             ser[meas]=np.NaN
#         return ser
# for p,i in df.index:
#     df.loc[p,i]['gradient_descent'] = empty_result(df.loc[p,i]['gradient_descent'])
# df


instance  \
p i                                             
1 0  <MaxCut.MaxCut object at 0x7f8c49901480>   
  1  <MaxCut.MaxCut object at 0x7f8c48e55600>   
  2  <MaxCut.MaxCut object at 0x7f8c42e7a230>   
  3  <MaxCut.MaxCut object at 0x7f8c42e780a0>   
  4  <MaxCut.MaxCut object at 0x7f8c42e7a800>   
  5  <MaxCut.MaxCut object at 0x7f8c42e7a6e0>   
2 0  <MaxCut.MaxCut object at 0x7f8c49901480>   
  1  <MaxCut.MaxCut object at 0x7f8c48e55600>   
  2  <MaxCut.MaxCut object at 0x7f8c42e7a230>   
  3  <MaxCut.MaxCut object at 0x7f8c42e780a0>   
  4  <MaxCut.MaxCut object at 0x7f8c42e7a800>   
  5  <MaxCut.MaxCut object at 0x7f8c42e7a6e0>   
3 0  <MaxCut.MaxCut object at 0x7f8c49901480>   
  1  <MaxCut.MaxCut object at 0x7f8c48e55600>   
  2  <MaxCut.MaxCut object at 0x7f8c42e7a230>   
  3  <MaxCut.MaxCut object at 0x7f8c42e780a0>   
  4  <MaxCut.MaxCut object at 0x7f8c42e7a800>   
  5  <MaxCut.MaxCut object at 0x7f8c42e7a6e0>   
4 0  <MaxCut.MaxCut object at 0x7f8c49901480>   
  1  <MaxCut.MaxCut object at 0x7f8c48e55600>   
  2  <MaxCut.MaxCut object at 0x7f8c42e7a230>   
  3  <MaxCut.MaxCut object at 0x7f8c42e780a0>   
  4  <MaxCut.MaxCut object at 0x7f8c42e7a800>   
  5  <MaxCut.MaxCut object at 0x7f8c42e7a6e0>   
5 0  <MaxCut.MaxCut object at 0x7f8c49901480>   
  1  <MaxCut.MaxCut object at 0x7f8c48e55600>   
  2  <MaxCut.MaxCut object at 0x7f8c42e7a230>   
  3  <MaxCut.MaxCut object at 0x7f8c42e780a0>   
  4  <MaxCut.MaxCut object at 0x7f8c42e7a800>   
  5  <MaxCut.MaxCut object at 0x7f8c42e7a6e0>   

                                                  tdvp  \
p i                                                      
1 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   
2 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   
3 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   
4 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   
5 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   

                                                 scipy  \
p i                                                      
1 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   
2 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                  

In [60]:
df.loc[1,2]['gradient_descent']

instance                  <MaxCut.MaxCut object at 0x7f8c42e785b0>
p                                                                1
n                                                                4
delta_0                                                     (1, 1)
tollarance                                                    0.01
algorithm                                         gradient_descent
res              \n        gradient_descent with general gradie...
delta                      (0.8931013871217299, 2.050579239575411)
value                                                    -0.474901
path             [(1.0196731297918973, 1.0315809822737378), (1....
steps                                                          161
num_fun_calls                                                 None
real duration                                            14.574935
message          Gradient descent solver terminated with       ...
Name: gradient_descent, dtype: object

In [35]:
tdvp_gd_n4['scipy'].loc[3,1]

instance         NaN
p                NaN
n                NaN
delta_0          NaN
tollarance       NaN
algorithm        NaN
res              NaN
delta            NaN
value            NaN
path             NaN
steps            NaN
num_fun_calls    NaN
real duration    NaN
message          NaN
Name: scipy, dtype: object

In [22]:
#scipy has no number set for 'steps' but we will use this field for computations
df = tdvp_gd_n5
for p,i in df.index:
    if df['scipy'].loc[p,i]['num_fun_calls'] != np.NaN: 
        df['scipy'].loc[p,i]['steps'] = df['scipy'].loc[p,i]['num_fun_calls']
    for alg in {'scipy', 'tdvp', 'gradient_descent'}:
        if df[alg].loc[p,i] is not np.NaN:
            res = df[alg].loc[p,i]
            try:
                res['success'] = res.res.success
            except  AttributeError:
                res['success'] = False
df['gradient_descent'].loc[1,1]

algorithm                                         gradient_descent
delta                      (1.574612801784724, 3.8499926307823387)
delta_0                                                     (1, 1)
instance                  <MaxCut.MaxCut object at 0x7fbada1952a0>
message          Gradient descent solver terminated with       ...
n                                                                5
num_fun_calls                                                 None
p                                                                1
path             [(1.116227916180378, 3.9142870339171623), (1.1...
real duration                                             8.669946
res              \n        gradient_descent with general gradie...
steps                                                           55
tollarance                                                       0
value                                                    -0.000082
success                                                      F

In [72]:
# df['gradient_descent'].loc[1,0].res

In [73]:
# with open('.//results//n4_p1.p','rb') as file:
#     df = pickle.load(file)

In [74]:
# with open("..//benchmarks//RK45_n10_p-1-5_Delta-1.p","rb") as file:
#     data: Benchmark = pickle.load(file)
#
# scipy_results = list()
# for d in data.results:
#     items = d.get("tdvp").__dict__.items()
#     for k,v in items:
#         d[k]=v
#         d["algorithm"] = "tdvp"
#     d["n"] = d["_qaoa"].n
#     dd = d.copy()
#     items = dd.get("scipy").__dict__.items()
#     for k,v in items:
#         dd[k]=v
#         dd["algorithm"] = "scipy"
#     dd["n"] = dd["_qaoa"].n
#     scipy_results.append(dd)
#
# data.results.extend(scipy_results)
#
# df = pd.DataFrame(data.results)
# df.pop("tdvp")
# df.pop("scipy")
# cols = df.columns.tolist()
# cols = cols[-1:] + cols[:-1]
# df = df[cols]
#
# df['steps'] = df.apply(lambda row: int(row['_num_steps']) if row.algorithm=="tdvp" else row['_num_fun_calls'], axis=1)
#
# df.insert(2,"algorithm",df.pop("algorithm"))
# df.insert(3,"_optimal_fun_value",df.pop("_optimal_fun_value"))
# df.insert(4,'steps', df.pop('steps'))
# df.insert(6,"_prob",df.pop("_prob"))
#
#
# df_tdvp = df[df.algorithm.eq("tdvp")]
# df_scipy = df[df.algorithm.eq("scipy")]
#
# df.rename(columns={"_optimal_fun_value":"C", "_prob":"prob", '_duration':'duration', '_optimal_state':'final_state', '_optimal_parameters':'final_parameters', '_num_gates':'num_gates','_qaoa':'qaoa'}, inplace=True)

fix error when LinAlgerror exception occured and thererfore QAOAResult.duration = None leads to an error in `f"{self.duration:.2f}"`

In [75]:
# for (p,i) in df.index:
#     try:
#         df.loc[p,i].__repr__()
#     except TypeError:
#         print((p,i))
#         (p_er,i_er) = (p,i)
# df['tdvp'].loc[p_er,i_er].res.value = np.NAN
# df['tdvp'].loc[p_er,i_er].res.num_fun_calls = np.NAN
# df['tdvp'].loc[p_er,i_er].res.num_steps = np.NAN
# df['tdvp'].loc[p_er,i_er].res.parameters = list(np.NAN for _ in range(6))
# df['tdvp'].loc[p_er,i_er].res.optimizer_name = 'tdvp'
# df['tdvp'].loc[p_er,i_er].res.duration = np.NAN

# Compute performance measures

In [76]:
# dfp = df[df['p']==1]
# measures = ['expected approximation ratio', 'groundspace overlap', 'groundspace sharpness', 'Approximation ratio', 'circuit count', 'gate count']
# types = ['min','max', 'mean', 'stdv']
# measure_index=pd.MultiIndex.from_product([measures, types])
# summary = pd.DataFrame(columns=measure_index)
# summary

## basic functions

In [18]:
def prob(num:int, state:Qobj):
    n = state.shape[0]
    return abs(state.overlap(basis(n,num)))**2

def answer(state:Qobj)->int:
    return max(range(state.shape[0]), key=lambda x:prob(x,state))

def bin_answer(answer:int, n:int)->str:
    return format(answer,  f'0{n}b')

## approximation ratio

In [19]:
def best_cost_value(qaoa:QAOA):
    return qaoa.H.groundstate()[0]

def approximation_ratio(bin_answer:int, best_cost_value:float, qaoa:QAOA):
    i = list(int(b) for b in bin_answer)
    dims = [2 for _ in range(qaoa.n)]
    return expect(qaoa.H,basis(dims,i))/best_cost_value

def expectation_of_approximation_ratio(state:Qobj, best_cost_value:float, qaoa:QAOA):
    return (expect(qaoa.H, state))/best_cost_value


## time measures

In [101]:
path_example = df['tdvp'].loc[1,0].path

In [102]:
path_example

[(1.0, 1.0),
 (1.0037607577294156, 0.9517213031907354),
 (0.9848448645981521, 0.6804949497295638),
 (0.8965704260352606, 0.5477886348697784),
 (0.8387316240369231, 0.4868805983967893),
 (0.8031554020697638, 0.45968337785686336),
 (0.7866942613160802, 0.4504589555059794),
 (0.7833300140465765, 0.4487310359568798)]

In [20]:
def num_metric_gates(n:int,p:int,qubo:NDArray):
    Q = np.count_nonzero(qubo)
    N = 2*p*(n+Q)
    C1 = 1/2 * n**4 + 2*n**3 + 7/2*n**2
    C2 = n**4 + 5/2*(n**3 + n**2)
    return N*C1*(7/3*p**2 +2*p + 1/3) + (2*C2-N*C1)*(1/2*p**2 +3/2*p)

def num_gradient_gates(n:int,p:int,qubo:NDArray):
    Q = np.count_nonzero(qubo)
    N = 2*p*(n+Q)
    return (n**4 -2*n**3 +n**2)/4 * (2*N+4) + 3*n*(n*(n-1))/2*(2*(N+3)) + 2*n**2*(2*N+3)

def num_qaoa_gates(n:int,p:int, qubo:NDArray):
    Q = np.count_nonzero(qubo)
    return 2*p*(n+Q)

def circuit_count(steps:int, algorithm:str, p:int, n:int):
    if algorithm =='tdvp':
        return steps*(p**2 + p)*(1/8*n**4 +1/2*n**3 -5/8*n**2) + 1
    if algorithm =="scipy":
        return steps + 1
    if algorithm=='gradient_descent':
        return 4*p*steps + 1

def gate_count(steps: int, algorithm: str, p: int, n: int, qaoa: QAOA):
    if algorithm=='tdvp':
        return steps*(num_metric_gates(n,p,qaoa.qubo) + num_gradient_gates(n,p,qaoa.qubo)) + num_qaoa_gates(n,p,qaoa.qubo)
    if algorithm=='scipy':
        return (1+steps)*num_qaoa_gates(n,p,qaoa.qubo)
    if algorithm=='gradient_descent':
        return (1+4*p*steps)*num_qaoa_gates(n,p,qaoa.qubo)

def path_length(path:List[Tuple[float]])->float:
    return sum(np.linalg.norm(np.array(path[i])-np.array(path[i+1])) for i in range(len(path)-1))

## Quality measures

In [21]:
# GROUNDSPACE IS NOW PART OF QAOA CLASS
# def groundspace(qaoa:QAOA):
#     eigenenergies, eigenstates = qaoa.H.eigenstates(sort='low')
#     groundenergy = eigenenergies[0]
#
#     num_of_eigenstates = 0
#     for e in eigenenergies:
#         if e==groundenergy:
#             num_of_eigenstates +=1
#         if e != groundenergy:
#             break
#     return eigenstates[:num_of_eigenstates]

def groundspace_overlap(groundstates:List[Qobj], state:Qobj):
    return  sum(abs(g.overlap(state))**2 for g in groundstates)

def groundspace_sharpness(groundstates:List[Qobj], state:Qobj):
    P_0 = sum(g*g.dag() for g in groundstates)
    return math.sqrt(variance(P_0,state))


## Compute everything

In [22]:
def compute_measures(res:pd.Series, qaoa:QAOA)->pd.Series:
    if not np.isnan(res.delta).any():
        state = qaoa.state(res.delta)
        res['answer'] = answer(state)
        res['bin_answer'] = bin_answer(res['answer'], res['n'])
        res['best_cost_value'] = best_cost_value(qaoa)
        res['approximation ratio'] = approximation_ratio(res['bin_answer'], res['best_cost_value'], qaoa)
        res['expectation approximation ratio']=expectation_of_approximation_ratio(state, res['best_cost_value'], qaoa)
        res['circuit count'] = circuit_count(res['steps'], res['algorithm'], res['p'], res['n'])
        res['gate count'] = gate_count(res['steps'], res['algorithm'], res['n'],res['p'], qaoa)
        res['path length'] = path_length(res['path'])
        res['groundspace overlap'] = groundspace_overlap(qaoa.groundspace, state)
        res['groundspace sharpness'] = groundspace_sharpness(qaoa.groundspace, state)
    else:
        res['answer'] = np.NaN
        res['bin_answer'] = np.NaN
        res['best_cost_value'] = np.NaN
        res['approximation ratio'] = np.NaN
        res['expectation approximation ratio']=np.NaN
        res['circuit count'] = np.NaN
        res['gate count'] = np.NaN
        res['path length'] = np.NaN
        res['groundspace overlap'] = np.NaN
        res['groundspace sharpness'] = np.NaN
    return res


In [23]:
def measures_mapping(row:pd.Series):
    qaoa = QAOA(row['instance'].qubo, p=row.name[0])
    if not np.isnan(row.tdvp.delta).any():
        row.tdvp = compute_measures(row.tdvp,qaoa)
    if not np.isnan(row.scipy.delta).any():
        row.scipy = compute_measures(row.scipy,qaoa)
    if not np.isnan(row.gradient_descent.delta).any():
        row.gradient_descent = compute_measures(row.gradient_descent,qaoa)
    # if row.scipy.algorithm is not None:
    #     row.scipy =compute_measures(row.scipy,qaoa)
    # if row.gradient_descent.algorithm is not None:
    #     row.gradient_descent =compute_measures(row.gradient_descent,qaoa)
    return row
def compute_measures_apply(df:pd.DataFrame)->pd.DataFrame:
    return df.apply(measures_mapping, axis=1)

In [31]:
df = compute_measures_apply(data)
df.head(10)

Pandas Apply:   0%|          | 0/74 [00:00<?, ?it/s]

(3, 1, 0)
(3, 1, 1)
(3, 2, 0)
(3, 2, 1)
(4, 1, 0)
(4, 1, 1)
(4, 1, 2)
(4, 1, 3)
(4, 1, 4)
(4, 1, 5)
(4, 2, 0)
(4, 2, 1)
(4, 2, 2)
(4, 2, 3)
(4, 2, 4)
(4, 2, 5)
(4, 3, 0)
(4, 3, 1)
(4, 3, 2)
(4, 3, 3)
(4, 3, 4)
(4, 3, 5)
(4, 4, 0)
(4, 4, 1)
(4, 4, 2)
(4, 4, 3)
(4, 4, 4)
(4, 4, 5)
(4, 5, 0)
(4, 5, 1)
(4, 5, 2)
(4, 5, 3)
(4, 5, 4)
(4, 5, 5)
(5, 1, 0)
(5, 1, 1)
(5, 1, 2)
(5, 1, 3)
(5, 1, 4)
(5, 1, 5)
(5, 1, 6)
(5, 1, 7)
(5, 1, 8)
(5, 1, 9)
(5, 2, 0)
(5, 2, 1)
(5, 2, 2)
(5, 2, 3)
(5, 2, 4)
(5, 2, 5)
(5, 2, 6)
(5, 2, 7)
(5, 2, 8)
(5, 2, 9)
(5, 3, 0)
(5, 3, 1)
(5, 3, 2)
(5, 3, 3)
(5, 3, 4)
(5, 3, 5)
(5, 3, 6)
(5, 3, 7)
(5, 3, 8)
(5, 3, 9)
(5, 4, 0)
(5, 4, 1)
(5, 4, 2)
(5, 4, 3)
(5, 4, 4)
(5, 4, 5)
(5, 4, 6)
(5, 4, 7)
(5, 4, 8)
(5, 4, 9)


instance  \
n p i                                             
3 1 0  <MaxCut.MaxCut object at 0x7f069a2e0df0>   
    1  <MaxCut.MaxCut object at 0x7f069a78cc70>   
  2 0  <MaxCut.MaxCut object at 0x7f069a2e0df0>   
    1  <MaxCut.MaxCut object at 0x7f069a78cc70>   
4 1 0  <MaxCut.MaxCut object at 0x7f069a78d150>   
    1  <MaxCut.MaxCut object at 0x7f069a78d2a0>   
    2  <MaxCut.MaxCut object at 0x7f069a9dc070>   
    3  <MaxCut.MaxCut object at 0x7f069a9dc0d0>   
    4  <MaxCut.MaxCut object at 0x7f069a9dc370>   
    5  <MaxCut.MaxCut object at 0x7f069a38f040>   

                                                    tdvp  \
n p i                                                      
3 1 0  instance                                    <M...   
    1  instance                                    <M...   
  2 0  instance                                    <M...   
    1  instance                                    <M...   
4 1 0  instance                                    <M...   
    1  instance                                    <M...   
    2  instance                                    <M...   
    3  instance                                    <M...   
    4  instance                                    <M...   
    5  instance                                    <M...   

                                                   scipy  \
n p i                                                      
3 1 0  instance                                    <M...   
    1  instance                                    <M...   
  2 0  instance                                    <M...   
    1  instance                                    <M...   
4 1 0  instance                                    <M...   
    1  instance                                    <M...   
    2  instance                                    <M...   
    3  instance                                    <M...   
    4  instance                                    <M...   
    5  instance                                    <M...   

                                        gradient_descent  
n p i                                                     
3 1 0  instance                                    <M...  
    1  instance                                    <M...  
  2 0  instance                                    <M...  
    1  instance                                    <M...  
4 1 0  instance                                    <M...  
    1  instance                                    <M...  
    2  instance                                    <M...  
    3  instance                                    <M...  
    4  instance                                    <M...  
    5  instance                                    <M...

In [37]:
df.xs(3,level='n')

instance  \
p i                                             
1 0  <MaxCut.MaxCut object at 0x7f069a2e0df0>   
  1  <MaxCut.MaxCut object at 0x7f069a78cc70>   
2 0  <MaxCut.MaxCut object at 0x7f069a2e0df0>   
  1  <MaxCut.MaxCut object at 0x7f069a78cc70>   

                                                  tdvp  \
p i                                                      
1 0  instance                                    <M...   
  1  instance                                    <M...   
2 0  instance                                    <M...   
  1  instance                                    <M...   

                                                 scipy  \
p i                                                      
1 0  instance                                    <M...   
  1  instance                                    <M...   
2 0  instance                                    <M...   
  1  instance                                    <M...   

                                      gradient_descent  
p i                                                     
1 0  instance                                    <M...  
  1  instance                                    <M...  
2 0  instance                                    <M...  
  1  instance                                    <M...

# Compute the statistics

In [112]:
# get one measure for all instances of one p and one algo
df.xs(1, level='p')['tdvp'].apply(lambda res: res['expectation approximation ratio'])

i
0    0.858965
1    0.963212
Name: tdvp, dtype: float64

In [118]:
n=5
df = tdvp_gd

In [167]:
min_p = min(df.xs(n,level='n').index, key= lambda x: x[0])[0]
max_p = max(df.xs(n,level='n').index, key= lambda x: x[0])[0]

algs = ['tdvp','scipy','gradient_descent']
index = pd.MultiIndex.from_product([range(3,6),algs,range(min_p, max_p+1)], names=['n','algorithm', 'p'])

measures = ['expectation approximation ratio', 'groundspace overlap', 'groundspace sharpness', 'approximation ratio', 'circuit count', 'gate count', 'num_fun_calls', 'steps', 'path length']
types = ['mean', 'std','min','max', ]
measure_index=pd.MultiIndex.from_product([measures, types])

stat = pd.DataFrame(index=index, columns=measure_index);stat

expectation approximation ratio                 \
                                                mean  std  min  max   
n algorithm        p                                                  
3 tdvp             1                             NaN  NaN  NaN  NaN   
                   2                             NaN  NaN  NaN  NaN   
                   3                             NaN  NaN  NaN  NaN   
                   4                             NaN  NaN  NaN  NaN   
  scipy            1                             NaN  NaN  NaN  NaN   
                   2                             NaN  NaN  NaN  NaN   
                   3                             NaN  NaN  NaN  NaN   
                   4                             NaN  NaN  NaN  NaN   
  gradient_descent 1                             NaN  NaN  NaN  NaN   
                   2                             NaN  NaN  NaN  NaN   
                   3                             NaN  NaN  NaN  NaN   
                   4                             NaN  NaN  NaN  NaN   
4 tdvp             1                             NaN  NaN  NaN  NaN   
                   2                             NaN  NaN  NaN  NaN   
                   3                             NaN  NaN  NaN  NaN   
                   4                             NaN  NaN  NaN  NaN   
  scipy            1                             NaN  NaN  NaN  NaN   
                   2                             NaN  NaN  NaN  NaN   
                   3                             NaN  NaN  NaN  NaN   
                   4                             NaN  NaN  NaN  NaN   
  gradient_descent 1                             NaN  NaN  NaN  NaN   
                   2                             NaN  NaN  NaN  NaN   
                   3                             NaN  NaN  NaN  NaN   
                   4                             NaN  NaN  NaN  NaN   
5 tdvp             1                             NaN  NaN  NaN  NaN   
                   2                             NaN  NaN  NaN  NaN   
                   3                             NaN  NaN  NaN  NaN   
                   4                             NaN  NaN  NaN  NaN   
  scipy            1                             NaN  NaN  NaN  NaN   
                   2                             NaN  NaN  NaN  NaN   
                   3                             NaN  NaN  NaN  NaN   
                   4                             NaN  NaN  NaN  NaN   
  gradient_descent 1                             NaN  NaN  NaN  NaN   
                   2                             NaN  NaN  NaN  NaN   
                   3                             NaN  NaN  NaN  NaN   
                   4                             NaN  NaN  NaN  NaN   

                     groundspace overlap                groundspace sharpness  \
                                    mean  std  min  max                  mean   
n algorithm        p                                                            
3 tdvp             1                 NaN  NaN  NaN  NaN                   NaN   
                   2                 NaN  NaN  NaN  NaN                   NaN   
                   3                 NaN  NaN  NaN  NaN                   NaN   
                   4                 NaN  NaN  NaN  NaN                   NaN   
  scipy            1                 NaN  NaN  NaN  NaN                   NaN   
                   2                 NaN  NaN  NaN  NaN                   NaN   
                   3                 NaN  NaN  NaN  NaN                   NaN   
                   4                 NaN  NaN  NaN  NaN                   NaN   
  gradient_descent 1                 NaN  NaN  NaN  NaN                   NaN   
                   2                 NaN  NaN  NaN  NaN                   NaN   
                   3                 NaN  NaN  NaN  NaN                   NaN   
                   4                 NaN  NaN  NaN  NaN                   NaN   
4 tdvp             1                

In [158]:
stat['path length']

mean  std  min  max
algorithm        p                    
gradient_descent 1  NaN  NaN  NaN  NaN
                 2  NaN  NaN  NaN  NaN
                 3  NaN  NaN  NaN  NaN
                 4  NaN  NaN  NaN  NaN

In [164]:
data.xs(1, level='p').xs(3,level='n')

,instance,tdvp,scipy,gradient_descent
i,,,,
0,<MaxCut.MaxCut object at 0x7fbad3715720>,instance <M...,instance <M...,instance <M...
1,<MaxCut.MaxCut object at 0x7fbad141ee90>,instance <M...,instance <M...,instance <M...


In [24]:
def compute_statistics(stat_row:pd.Series, data:pd.DataFrame):
    # LOOP THROUGH ALL STATISTICS
    measures = ['expectation approximation ratio', 'groundspace overlap', 'groundspace sharpness', 'approximation ratio', 'circuit count', 'gate count', 'num_fun_calls', 'steps', 'path length']
    
    n,algo, p = stat_row.name
    print(n,algo,p)
    for meas in measures:
        print(meas)
        # get measure for all instances of one p and one algo
        data_meas:pd.Series = data.xs(p, level='p').xs(n,level='n')[algo].apply(lambda res: res[meas])
        stat_row[meas,'mean'] = data_meas.mean()
        stat_row[meas,'std'] = data_meas.std()
        stat_row[meas,'min'] = data_meas.min()
        stat_row[meas,'max'] = data_meas.max()
    return stat_row

## Save the processed data

In [175]:
stats_new = stats.apply(lambda x: compute_statistics(x, data), axis=1)
stats_new

3 gradient_descent 1
expectation approximation ratio
groundspace overlap
groundspace sharpness
approximation ratio
circuit count
gate count
num_fun_calls
steps
path length
3 gradient_descent 2
expectation approximation ratio
groundspace overlap
groundspace sharpness
approximation ratio
circuit count
gate count
num_fun_calls
steps
path length
4 gradient_descent 1
expectation approximation ratio
groundspace overlap
groundspace sharpness
approximation ratio
circuit count
gate count
num_fun_calls
steps
path length
4 gradient_descent 2
expectation approximation ratio
groundspace overlap
groundspace sharpness
approximation ratio
circuit count
gate count
num_fun_calls
steps
path length
4 gradient_descent 3
expectation approximation ratio
groundspace overlap
groundspace sharpness
approximation ratio
circuit count
gate count
num_fun_calls
steps
path length
4 gradient_descent 4
expectation approximation ratio
groundspace overlap
groundspace sharpness
approximation ratio
circuit count
gate count


expectation approximation ratio                      \
                                                mean       std       min   
n Algorithm        p                                                       
3 gradient_descent 1                        0.911161  0.073687  0.859056   
                   2                        0.942691  0.046450  0.909846   
4 gradient_descent 1                        0.515058  0.413939  0.067843   
                   2                        0.910650  0.025242  0.874111   
                   3                             NaN       NaN       NaN   
                   4                             NaN       NaN       NaN   
                   5                             NaN       NaN       NaN   
5 gradient_descent 1                        0.254755  0.301218 -0.001730   
                   2                        0.701179  0.265325  0.513566   
                   3                             NaN       NaN       NaN   
                   4                             NaN       NaN       NaN   

                               groundspace overlap                      \
                           max                mean       std       min   
n Algorithm        p                                                     
3 gradient_descent 1  0.963266            0.867664  0.146380  0.764157   
                   2  0.975536            0.912558  0.098755  0.842728   
4 gradient_descent 1  0.963282            0.456997  0.394647  0.051365   
                   2  0.931966            0.864552  0.060204  0.797145   
                   3       NaN                 NaN       NaN       NaN   
                   4       NaN                 NaN       NaN       NaN   
                   5       NaN                 NaN       NaN       NaN   
5 gradient_descent 1  0.963282            0.134955  0.308172  0.002314   
                   2  0.888792            0.523556  0.320988  0.296583   
                   3       NaN                 NaN       NaN       NaN   
                   4       NaN                 NaN       NaN       NaN   

                               groundspace sharpness            ...  \
                           max                  mean       std  ...   
n Algorithm        p                                            ...   
3 gradient_descent 1  0.971170              0.295927  0.181864  ...   
                   2  0.982388              0.247797  0.164417  ...   
4 gradient_descent 1  0.961850              0.329895  0.135659  ...   
                   2  0.930363              0.333246  0.066633  ...   
                   3       NaN                   NaN       NaN  ...   
                   4       NaN                   NaN       NaN  ...   
                   5       NaN                   NaN       NaN  ...   
5 gradient_descent 1  0.952341              0.157729  0.091525  ...   
                   2  0.750528              0.444729  0.017001  ...   
                   3       NaN                   NaN       NaN  ...   
                   4       NaN                   NaN       NaN  ...   

                     num_fun_calls                 steps                     \
                               min      max         mean          std   min   
n Algorithm        p                                                          
3 gradient_descent 1         148.0    224.0    46.500000    13.435029  37.0   
                   2         528.0   8672.0   575.000000   719.834703  66.0   
4 gradient_descent 1         116.0  12000.0   562.166667  1195.262551  29.0   
                   2         288.0  24000.0  1123.333333  1457.722836  36.0   
                   3           NaN      NaN          NaN          NaN   NaN   
                   4           NaN      NaN          NaN          NaN   NaN   
                   5           NaN      NaN          NaN          NaN   NaN   
5 gradient_descent 1          96.0  12000.0   407.300000   917.833445  24.0   
                   2         456.0  24000.0  2416.40000

In [176]:
stats_new['path length']

mean       std       min       max
n Algorithm        p                                        
3 gradient_descent 1  0.650000  0.082290  0.591812  0.708187
                   2  1.333444  1.582315  0.214579  2.452310
4 gradient_descent 1  0.619332  0.303366  0.258439  1.076409
                   2  1.029225  0.602011  0.155999  1.535807
                   3       NaN       NaN       NaN       NaN
                   4       NaN       NaN       NaN       NaN
                   5       NaN       NaN       NaN       NaN
5 gradient_descent 1  0.460339  0.326492  0.095637  1.107871
                   2  0.845453  0.414154  0.552601  1.138304
                   3       NaN       NaN       NaN       NaN
                   4       NaN       NaN       NaN       NaN

In [61]:
data

instance  \
n p i                                             
3 1 0  <MaxCut.MaxCut object at 0x7f069a2e0df0>   
    1  <MaxCut.MaxCut object at 0x7f069a78cc70>   
  2 0  <MaxCut.MaxCut object at 0x7f069a2e0df0>   
    1  <MaxCut.MaxCut object at 0x7f069a78cc70>   
4 1 0  <MaxCut.MaxCut object at 0x7f069a78d150>   
...                                         ...   
5 4 5  <MaxCut.MaxCut object at 0x7f069a77eef0>   
    6  <MaxCut.MaxCut object at 0x7f069a77ead0>   
    7  <MaxCut.MaxCut object at 0x7f069a77d4e0>   
    8  <MaxCut.MaxCut object at 0x7f069a77d720>   
    9  <MaxCut.MaxCut object at 0x7f069a77de70>   

                                                    tdvp  \
n p i                                                      
3 1 0  instance                                    <M...   
    1  instance                                    <M...   
  2 0  instance                                    <M...   
    1  instance                                    <M...   
4 1 0  instance                                    <M...   
...                                                  ...   
5 4 5  instance                                    <M...   
    6  instance                                    <M...   
    7  instance                                    <M...   
    8  instance                                    <M...   
    9  instance                                    <M...   

                                                   scipy  \
n p i                                                      
3 1 0  instance                                    <M...   
    1  instance                                    <M...   
  2 0  instance                                    <M...   
    1  instance                                    <M...   
4 1 0  instance                                    <M...   
...                                                  ...   
5 4 5  instance                                    <M...   
    6  instance                                    <M...   
    7  instance                                    <M...   
    8  instance                                    <M...   
    9  instance                                    <M...   

                                        gradient_descent  
n p i                                                     
3 1 0  instance                                    <M...  
    1  instance                                    <M...  
  2 0  instance                                    <M...  
    1  instance                                    <M...  
4 1 0  instance                                    <M...  
...                                                  ...  
5 4 5  algorithm                          gradient_de...  
    6  algorithm                          gradient_de...  
    7  algorithm                          gradient_de...  
    8  algorithm                          gradient_de...  
    9  algorithm                          gradient_de...  

[74 rows x 4 columns]

In [177]:
if False:
    # # noinspection PyUnreachableCode
    with open('.//results//stats_tdvp_gd.p', 'wb') as file:
        pickle.dump(stats_new, file)
if False:
    with open('.//results//data.p', 'wb') as file:
        pickle.dump(data, file)

# OLD

In [ ]:
# # df['groundspace'] = df.swifter.apply(lambda row: groundspace(row.qaoa), axis=1)
# # df.insert(3,'groundspace_overlap', df.swifter.apply(lambda row: groundspace_overlap(row['groundspace'], row['final_state']), axis=1))
# # df.insert(4, 'groundspace_sharpness', df.swifter.apply(lambda row: groundspace_sharpness(row['groundspace'], row['final_state']), axis=1))
# df['steps']
# df['circuit_count']= df.swifter.apply(lambda row: circuit_count(row['steps'], row['algorithm'], row['p'], row['n']), axis=1)
# df['gate_count'] =df.swifter.apply(lambda row: gate_count(row['steps'], row['algorithm'], row['p'], row['n'],
#                                                           row['qaoa']), axis=1)
# df['answer'] = df.swifter.apply(lambda row: answer(row['final_state']), axis=1)
# df['bin_answer'] = df.swifter.apply(lambda row: bin_answer(row['final_state']), axis=1)
# df.insert(3, 'best_cost_value', df.swifter.apply(lambda row: best_cost_value(row['qaoa']), axis=1))
# df.insert(3, 'approximation_ratio', df.swifter.apply(lambda row: approximation_ratio(row['bin_answer'],row['best_cost_value'],row['qaoa']), axis=1))
# df.insert(3, 'expectation_of_approximation_ratio', df.swifter.apply(lambda row: expectation_of_approximation_ratio(row['final_state'],row['best_cost_value'],row['qaoa']), axis=1))
# with open('..//benchmarks//n10_data.p', 'wb') as file:
#     pickle.dump(df,file)

# Combine all into one

In [121]:
ns = (3,4,5)
tuples3 = [tuple([3,]+list(index)) for index in stats3.index.to_list()]
tuples4 = [tuple([4,]+list(index)) for index in stats4.index.to_list()]
tuples5 = [tuple([5,]+list(index)) for index in stats5.index.to_list()]
index = pd.MultiIndex.from_tuples(tuples3+tuples4+tuples5, names=['n','Algorithm','p'])
stat = pd.DataFrame(index=index, columns=stats5.columns)

In [122]:
stat

expectation approximation ratio                 \
                                                mean  std  min  max   
n Algorithm        p                                                  
3 gradient_descent 1                             NaN  NaN  NaN  NaN   
                   2                             NaN  NaN  NaN  NaN   
4 gradient_descent 1                             NaN  NaN  NaN  NaN   
                   2                             NaN  NaN  NaN  NaN   
                   3                             NaN  NaN  NaN  NaN   
                   4                             NaN  NaN  NaN  NaN   
                   5                             NaN  NaN  NaN  NaN   
5 gradient_descent 1                             NaN  NaN  NaN  NaN   
                   2                             NaN  NaN  NaN  NaN   
                   3                             NaN  NaN  NaN  NaN   
                   4                             NaN  NaN  NaN  NaN   

                     groundspace overlap                groundspace sharpness  \
                                    mean  std  min  max                  mean   
n Algorithm        p                                                            
3 gradient_descent 1                 NaN  NaN  NaN  NaN                   NaN   
                   2                 NaN  NaN  NaN  NaN                   NaN   
4 gradient_descent 1                 NaN  NaN  NaN  NaN                   NaN   
                   2                 NaN  NaN  NaN  NaN                   NaN   
                   3                 NaN  NaN  NaN  NaN                   NaN   
                   4                 NaN  NaN  NaN  NaN                   NaN   
                   5                 NaN  NaN  NaN  NaN                   NaN   
5 gradient_descent 1                 NaN  NaN  NaN  NaN                   NaN   
                   2                 NaN  NaN  NaN  NaN                   NaN   
                   3                 NaN  NaN  NaN  NaN                   NaN   
                   4                 NaN  NaN  NaN  NaN                   NaN   

                           ... gate count      num_fun_calls                 \
                      std  ...        min  max          mean  std  min  max   
n Algorithm        p       ...                                                
3 gradient_descent 1  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                   2  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
4 gradient_descent 1  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                   2  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                   3  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                   4  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                   5  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
5 gradient_descent 1  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                   2  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                   3  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                   4  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   

                     steps                 
                      mean  std  min  max  
n Algorithm        p                       
3 gradient_descent 1   NaN  NaN  NaN  NaN  
                   2   NaN  NaN  NaN  NaN  
4 gradient_descent 1   NaN  NaN  NaN  NaN  
                   2   NaN  NaN  NaN  NaN  
                   3   NaN  NaN  NaN  NaN  
                   4   NaN  NaN  NaN  NaN  
                   5   NaN  NaN  NaN  NaN  
5 gradient_descent 1   NaN  NaN  NaN  NaN  
                   2   NaN  NaN  NaN  NaN  
                   3   NaN  NaN  NaN  NaN  
                   4   NaN  NaN  NaN  NaN  

[11 rows x 32 columns]

In [197]:
tuples3

[('tdvp', 1),
 ('tdvp', 2),
 ('scipy', 1),
 ('scipy', 2),
 ('gradient_descent', 1),
 ('gradient_descent', 2)]

In [123]:
for n,a,i in stat.index:
    if n==3:
        source = stats3
    elif n==4:
        source = stats4
    else:
        source = stats5
    stat.loc[n,a,i] = source.loc[a,i]

In [124]:
stat

expectation approximation ratio                      \
                                                mean       std       min   
n Algorithm        p                                                       
3 gradient_descent 1                        0.306478  0.432672  0.000533   
                   2                        0.652124  0.047679   0.61841   
4 gradient_descent 1                         0.09143  0.109532 -0.008782   
                   2                       -0.005084  0.180493 -0.334642   
                   3                        0.023407  0.365324 -0.657712   
                   4                       -0.066938  0.250522 -0.483237   
                   5                       -0.076148  0.236878 -0.495442   
5 gradient_descent 1                        0.045288  0.054732  0.000008   
                   2                         0.07098  0.191743 -0.104134   
                   3                       -0.033717  0.153383 -0.337194   
                   4                       -0.001143  0.194539 -0.425301   

                               groundspace overlap                      \
                           max                mean       std       min   
n Algorithm        p                                                     
3 gradient_descent 1  0.612424            0.358776  0.307027  0.141675   
                   2  0.685838            0.636691  0.049783  0.601489   
4 gradient_descent 1  0.292312            0.119267  0.074892  0.041842   
                   2  0.137274            0.044903  0.033964  0.001868   
                   3  0.335188            0.195955  0.140206  0.016026   
                   4  0.292379            0.161946  0.187445  0.013606   
                   5  0.156692            0.120738   0.13446  0.002831   
5 gradient_descent 1  0.152901            0.032234  0.020318  0.001355   
                   2  0.598427            0.111338   0.13391  0.007602   
                   3  0.171506            0.058335  0.049508  0.005084   
                   4  0.242827            0.102955  0.124657  0.003596   

                               groundspace sharpness            ...  \
                           max                  mean       std  ...   
n Algorithm        p                                            ...   
3 gradient_descent 1  0.575877              0.421463  0.102879  ...   
                   2  0.671892              0.479558   0.01419  ...   
4 gradient_descent 1    0.2403              0.307251  0.084609  ...   
                   2  0.094225              0.187709  0.089605  ...   
                   3  0.347903              0.349153  0.152056  ...   
                   4  0.450245              0.291246  0.161052  ...   
                   5  0.355029              0.259517  0.168801  ...   
5 gradient_descent 1  0.059707              0.162219   0.07078  ...   
                   2  0.453147              0.259983  0.130007  ...   
                   3  0.157316              0.211632  0.093915  ...   
                   4  0.442111              0.258056  0.114392  ...   

                     gate count            num_fun_calls                 \
                            min        max          mean  std  min  max   
n Algorithm        p                                                      
3 gradient_descent 1    34032.0    42540.0           NaN  NaN  NaN  NaN   
                   2    61614.0    75306.0           NaN  NaN  NaN  NaN   
4 gradient_descent 1    77528.0   119816.0           NaN  NaN  NaN  NaN   
                   2   298080.0   447120.0           NaN  NaN  NaN  NaN   
                   3   911976.0  1332888.0           NaN  NaN  NaN  NaN   
                   4   148848.0   212640.0           NaN  NaN  NaN  NaN   
                   5   710520.0   994728.0           NaN  NaN  NaN  NaN   
5 gradient_descent 1   154140.0   286260.0           NaN  NaN  NaN  NaN   
                   2   552150.0   993870.0           NaN  NaN  NaN  NaN   
                   3   3

In [125]:
# with open('./results/stats_tdvp_gd.p','wb') as f:
#     pickle.dump(stat,f)

In [174]:
with open('./results/stats_tdvp_gd.p','rb')as f:
    stats = pickle.load(f)

In [136]:
stats.columns[-40:]

MultiIndex([('expectation approximation ratio', 'mean'),
            ('expectation approximation ratio',  'std'),
            ('expectation approximation ratio',  'min'),
            ('expectation approximation ratio',  'max'),
            (            'groundspace overlap', 'mean'),
            (            'groundspace overlap',  'std'),
            (            'groundspace overlap',  'min'),
            (            'groundspace overlap',  'max'),
            (          'groundspace sharpness', 'mean'),
            (          'groundspace sharpness',  'std'),
            (          'groundspace sharpness',  'min'),
            (          'groundspace sharpness',  'max'),
            (            'approximation ratio', 'mean'),
            (            'approximation ratio',  'std'),
            (            'approximation ratio',  'min'),
            (            'approximation ratio',  'max'),
            (                  'circuit count', 'mean'),
            (                  

In [248]:
scipy_vs_tdvp = data.apply(lambda row: abs(row['scipy']['path length']-row['tdvp']['path length']),axis=1)
gd_vs_tdvp = data.apply(lambda row: abs(row['tdvp']['path length']-row['gradient_descent']['path length']),axis=1)


In [256]:
max(gd_vs_tdvp.index, key=lambda idx: gd_vs_tdvp.loc[idx])

(3, 2, 0)

In [257]:
max(scipy_vs_tdvp.index, key=lambda idx: scipy_vs_tdvp.loc[idx])

(4, 5, 1)

In [258]:
scipy_vs_tdvp.loc[3,2,0]

12.607283068499388

In [6]:
with open("results/tdvp_vs_gd_n5_new.p",'rb') as file:
    tdvp_gd_n5 = pickle.load(file)
with open("results/tdvp_vs_gd_n4_new.p",'rb') as file:
    tdvp_gd_n4 = pickle.load(file)
with open("results/tdvp_vs_gd_n3_new.p",'rb') as file:
    tdvp_gd_n3 = pickle.load(file)

In [7]:
tdvp_gd_n4.loc[4,2]['gradient_descent']['delta']

(6.578019976471005,
 0.332951500230493,
 -0.8380960017938541,
 -1.2696793541832072,
 -4.412169604840367,
 7.618506132038959,
 1.525259839095904,
 2.5224527645518737)

In [77]:
np.isnan(tdvp_gd_n4.loc[4,2].scipy.delta).any()

True

In [69]:
compute_measures(tdvp_gd_n4.loc[4,2]['gradient_descent'], QAOA(tdvp_gd_n4.instance.loc[4,2].qubo,p=4))

algorithm                                                           gradient_descent
delta                              (6.578019976471005, 0.332951500230493, -0.8380...
delta_0                                                     (1, 1, 1, 1, 1, 1, 1, 1)
instance                                    <MaxCut.MaxCut object at 0x7fbad9e3c280>
message                            Gradient descent solver terminated with       ...
n                                                                                  4
num_fun_calls                                                                   None
p                                                                                  4
path                               [(0.9091624403035894, 0.8820339186522688, 1.45...
real duration                                                              24.382978
res                                \n        gradient_descent with general gradie...
steps                                                            

In [82]:
tdvp_gd_n3 = compute_measures_apply(tdvp_gd_n3)

In [80]:
tdvp_gd_n4.loc[4,2]['gradient_descent']

algorithm                                                           gradient_descent
delta                              (6.578019976471005, 0.332951500230493, -0.8380...
delta_0                                                     (1, 1, 1, 1, 1, 1, 1, 1)
instance                                    <MaxCut.MaxCut object at 0x7fbad9e3c280>
message                            Gradient descent solver terminated with       ...
n                                                                                  4
num_fun_calls                                                                   None
p                                                                                  4
path                               [(0.9091624403035894, 0.8820339186522688, 1.45...
real duration                                                              24.382978
res                                \n        gradient_descent with general gradie...
steps                                                            

In [106]:
with open("results/data.p",'wb') as file:
    data = pickle.load(file)

In [83]:
tdvp_gd = pd.DataFrame(index=data.index, columns=['tdvp','scipy','gradient_descent', ])

In [84]:
mydict = {3:tdvp_gd_n3, 4:tdvp_gd_n4, 5:tdvp_gd_n5}

In [85]:
for n,p,i in tdvp_gd.index:
    tdvp_gd.loc[n,p,i] = mydict[n].loc[p,i]

In [86]:
tdvp_gd.loc[4,1,0]['gradient_descent']

algorithm                                                           gradient_descent
delta                                      (0.6414109526554626, -1.8278606123421923)
delta_0                                                                       (1, 1)
instance                                    <MaxCut.MaxCut object at 0x7fbad9e3ff40>
message                            Gradient descent solver terminated with       ...
n                                                                                  4
num_fun_calls                                                                   None
p                                                                                  1
path                               [(0.8104951437483123, 1.5863995955479595), (0....
real duration                                                               7.922438
res                                \n        gradient_descent with general gradie...
steps                                                            

In [87]:
with open("results/tdvp_gd.p",'wb') as file:
    pickle.dump(tdvp_gd,file)